<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/GRU-intra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import GRU, Dense
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/intravehicle.csv')

In [4]:
y_data = data.pop('4')
x_data = data

In [5]:
train_x,test_x,train_y,test_y = train_test_split(x_data,y_data,test_size=0.2,random_state=0,shuffle=True)

In [6]:
train_x = train_x.reset_index(drop=True)
test_x = test_x.reset_index(drop=True) 
train_y = train_y.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)

In [7]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [8]:
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)

In [9]:
model = Sequential()
model.add(GRU(100,input_shape=(len(x_data.columns),1)))
model.add(Dense(4,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 100)               30900     
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 31,304
Trainable params: 31,304
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(train_x,train_y,epochs=6,batch_size=32)

Epoch 1/6
414237/414237 [==============================] - 1464s 4ms/step - loss: 0.0391 - accuracy: 0.9865
Epoch 2/6
414237/414237 [==============================] - 1464s 4ms/step - loss: 0.0264 - accuracy: 0.9910
Epoch 3/6
414237/414237 [==============================] - 1466s 4ms/step - loss: 0.0248 - accuracy: 0.9914
Epoch 4/6
414237/414237 [==============================] - 1471s 4ms/step - loss: 0.0239 - accuracy: 0.9916
Epoch 5/6
414237/414237 [==============================] - 1474s 4ms/step - loss: 0.0229 - accuracy: 0.9918
Epoch 6/6
414237/414237 [==============================] - 1434s 3ms/step - loss: 0.0213 - accuracy: 0.9920


In [11]:
score,accuracy = model.evaluate(test_x,test_y)
print('Score',score,"Accuracy",accuracy)

103560/103560 [==============================] - 248s 2ms/step - loss: 0.0188 - accuracy: 0.9921
Score 0.01878693513572216 Accuracy 0.9920507669448853
